In [1]:
pip install delta-spark==3.3.2 dotenv

  Using cached pyspark-3.5.7-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from datetime import datetime

In [3]:
from dotenv import load_dotenv
import os
load_dotenv('/opt/workspace/.env')
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT_DOCKER")
MINIO_ACCESS = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET = os.getenv("MINIO_ROOT_PASSWORD")

In [4]:
builder = (
    SparkSession.builder.appName("SilverToGold_Delta")
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
df = spark.read.format("delta").load("s3a://gold/dim_linha/")

duplicados = (
    df.groupBy("codigo_linha")
      .agg(F.countDistinct("letreiro").alias("n_letreiros"))
      .filter("n_letreiros > 1")
)

print("🔎 Linhas com mais de um letreiro para o mesmo código:")
duplicados.show()

🔎 Linhas com mais de um letreiro para o mesmo código:
+------------+-----------+
|codigo_linha|n_letreiros|
+------------+-----------+
+------------+-----------+



In [6]:
duplicados = (
    df.groupBy("letreiro")
      .agg(F.countDistinct("codigo_linha").alias("n_codigos"))
      .filter("n_codigos > 1")
)

print("🔎 Letreiros que correspondem a mais de um código:")
duplicados.show()

🔎 Letreiros que correspondem a mais de um código:
+--------+---------+
|letreiro|n_codigos|
+--------+---------+
+--------+---------+



In [7]:
df.groupBy("codigo_linha").agg(F.collect_set("letreiro").alias("letreiros")).show(10, False)


+------------+---------+
|codigo_linha|letreiros|
+------------+---------+
|1           |[5119-10]|
|4           |[669A-10]|
|5           |[6801-10]|
|6           |[6837-10]|
|8           |[695V-10]|
|12          |[6835-10]|
|13          |[6836-10]|
|14          |[6825-10]|
|16          |[6826-10]|
|19          |[6820-10]|
+------------+---------+
only showing top 10 rows



In [8]:
# Paths
today = "data_ref=" + datetime.today().strftime('%Y-%m-%d')
SILVER_POSICAO = "s3a://silver/posicao/"
GOLD_DIM_LINHA = "s3a://gold/dim_linha/"
GOLD_FATO_POSICAO = "s3a://gold/fato_posicao/"

print(f"📂 Lendo Silver: {SILVER_POSICAO}")
print(f"📂 Lendo Dim Linha: {GOLD_DIM_LINHA}")

# Leitura
posicao = spark.read.format("delta").load(SILVER_POSICAO)
dim_linha = spark.read.format("delta").load(GOLD_DIM_LINHA)

📂 Lendo Silver: s3a://silver/posicao/
📂 Lendo Dim Linha: s3a://gold/dim_linha/


In [9]:
# Join com dim linha pra manter linhas que existem na dimensão
fato = (
    posicao.alias("p")
    .join(dim_linha.alias("l"), "codigo_linha", "inner")
    .select(
        F.col("p.codigo_veiculo"),
        F.col("p.codigo_linha"),
        F.col("p.sentido"),
        F.col("p.acessibilidade"),
        F.col("p.latitude"),
        F.col("p.longitude"),
        F.col("p.hora_referencia"),
        F.col("p.ultima_atualizacao"),
        F.col("p.data_ref"),
        F.current_timestamp().alias("transform_gold_timestamp")
    )
)

fato.show(20)

+--------------+------------+-------+--------------+----------+----------+-------------------+-------------------+----------+------------------------+
|codigo_veiculo|codigo_linha|sentido|acessibilidade|  latitude| longitude|    hora_referencia| ultima_atualizacao|  data_ref|transform_gold_timestamp|
+--------------+------------+-------+--------------+----------+----------+-------------------+-------------------+----------+------------------------+
|         10002|        2450|      1|          true|-23.510699|-46.705584|2025-11-07 19:15:00|2025-11-07 22:15:25|2025-11-07|    2025-11-12 04:17:...|
|         10002|        2450|      1|          true|-23.510699|-46.705584|2025-11-07 19:15:00|2025-11-07 22:15:25|2025-11-07|    2025-11-12 04:17:...|
|         10002|        2450|      1|          true|-23.510699|-46.705584|2025-11-07 19:15:00|2025-11-07 22:15:25|2025-11-07|    2025-11-12 04:17:...|
|         10002|        2450|      1|          true|-23.510699|-46.705584|2025-11-07 19:15:00|

In [10]:
# Salvar 
"""
(
    fato.write
    .format("delta")
    .mode("overwrite")  # ou "append" se quiser manter histórico incremental
    .partitionBy("data_ref")
    .save(GOLD_FATO_POSICAO)
)
print(f"fato_posicao salvo em {GOLD_FATO_POSICAO}")
"""

'\n(\n    fato.write\n    .format("delta")\n    .mode("overwrite")  # ou "append" se quiser manter histórico incremental\n    .partitionBy("data_ref")\n    .save(GOLD_FATO_POSICAO)\n)\nprint(f"fato_posicao salvo em {GOLD_FATO_POSICAO}")\n'

In [11]:
fato.groupBy("data_ref") \
    .agg(
        F.count("*").alias("total_registros"),
        F.countDistinct("codigo_veiculo").alias("onibus_unicos"),
        F.countDistinct("codigo_linha").alias("linhas_ativas")
    ).orderBy("data_ref", ascending=False).show()


+----------+---------------+-------------+-------------+
|  data_ref|total_registros|onibus_unicos|linhas_ativas|
+----------+---------------+-------------+-------------+
|2025-11-12|          34902|         2075|          545|
|2025-11-11|        1752894|         6310|          749|
|2025-11-10|         817206|         6263|          785|
|2025-11-09|             36|            1|            1|
|2025-11-08|         150882|         4266|          730|
|2025-11-07|         573126|         5276|          612|
|2025-11-05|             24|            1|            1|
+----------+---------------+-------------+-------------+



In [12]:
fato.groupBy("data_ref").agg(
    F.min("hora_referencia").alias("inicio_dia"),
    F.max("hora_referencia").alias("fim_dia"),
    F.count("*").alias("total_registros"),
    F.countDistinct("codigo_veiculo").alias("onibus_unicos"),
    F.countDistinct("codigo_linha").alias("linhas_ativas"),
).orderBy("data_ref", ascending=False).show(truncate=False)

+----------+-------------------+-------------------+---------------+-------------+-------------+
|data_ref  |inicio_dia         |fim_dia            |total_registros|onibus_unicos|linhas_ativas|
+----------+-------------------+-------------------+---------------+-------------+-------------+
|2025-11-12|2025-11-12 22:11:00|2025-11-12 22:14:00|34902          |2075         |545          |
|2025-11-11|2025-11-11 00:00:00|2025-11-12 22:14:00|1752894        |6310         |749          |
|2025-11-10|2025-11-10 01:11:00|2025-11-11 14:34:00|817206         |6263         |785          |
|2025-11-09|2025-11-11 00:56:00|2025-11-11 01:16:00|36             |1            |1            |
|2025-11-08|2025-11-08 02:52:00|2025-11-10 01:36:00|150882         |4266         |730          |
|2025-11-07|2025-11-07 19:15:00|2025-11-08 03:52:00|573126         |5276         |612          |
|2025-11-05|2025-11-10 11:16:00|2025-11-10 11:26:00|24             |1            |1            |
+----------+------------------

In [13]:
spark.stop()